In [1]:
import os
import csv
import math

EXPERIMENTS_DIR = "../experiments"
METRIC_KEYS = {
    "Average Best Loss": "Loss",
    "Std Best Loss": "Loss",
    "Average Gradient Time": "Grad (s)",
    "Std Gradient Time": "Grad (s)",
    "Average Sampling Time": "Sampling (s)",
    "Std Sampling Time": "Sampling (s)",
    "Average PGD Time": "PGD (s)",
    "Std PGD Time": "PGD (s)",
    "Average Loss Time": "LossTime (s)",
    "Std Loss Time": "LossTime (s)",
    "Average Total Time": "Total (s)",
    "Std Total Time": "Total (s)",
}
TIME_METRICS = [
    ("Average Gradient Time", "Std Gradient Time"),
    ("Average Sampling Time", "Std Sampling Time"),
    ("Average PGD Time", "Std PGD Time"),
    ("Average Loss Time", "Std Loss Time"),
    ("Average Total Time", "Std Total Time"),
]


def latex_escape(text: str) -> str:
    return text.replace("_", r"\_").replace("%", r"\%")


def format_numeric(value: str, param: str = "") -> str:
    if value in ("True", "False"):
        return value
    if param.lower() in {
        "seed",
        "iter",
        "search_width",
        "min_search_width",
        "num_steps",
        "num_prompts",
        "k",
    }:
        try:
            return str(int(float(value)))
        except ValueError:
            return latex_escape(value)
    try:
        return f"{float(value):.4f}"
    except ValueError:
        return latex_escape(value)


def combine_avg_std(avg: str, std: str) -> str:
    """
    Convert two strings avg/std into a formatted 'a±s' string.
    Treat any missing or NaN values as 0.0.
    """
    try:
        a = float(avg) if avg not in (None, "") else 0.0
        s = float(std) if std not in (None, "") else 0.0
        # if either is nan, treat as zero
        if math.isnan(a):
            a = 0.0
        if math.isnan(s):
            s = 0.0
        return f"{a:.4f}±{s:.4f}"
    except ValueError:
        return "0.0000±0.0000"


def collect_rows():
    params_list, summary_list = [], []
    all_params, as_cols = set(), set()
    for idx, folder in enumerate(sorted(os.listdir(EXPERIMENTS_DIR)), start=1):
        path = os.path.join(EXPERIMENTS_DIR, folder)
        if not os.path.isdir(path):
            continue
        exp_id = f"exp{idx}"
        params = {"Experiment": exp_id}
        pfile = os.path.join(path, "parameters.csv")
        if os.path.exists(pfile):
            with open(pfile, newline="") as f:
                for row in csv.DictReader(f):
                    k, v = row["Parameter"], row["Value"]
                    if k != "debug_output":
                        params[k] = v
                        all_params.add(k)
        params_list.append(params)

        metrics = {}
        summary_file = os.path.join(path, "summary.csv")
        if os.path.exists(summary_file):
            with open(summary_file, newline="") as f:
                for row in csv.DictReader(f):
                    metrics[row["Metric"]] = row["Value"]

        as_map = {}
        for d in os.listdir(path):
            if d.startswith("evaluation_k"):
                k = d.split("_k", 1)[1]
                csv_path = os.path.join(path, d, "summary.csv")
                if not os.path.exists(csv_path):
                    continue
                succ = tot = 0
                with open(csv_path, newline="") as f:
                    for row in csv.DictReader(f):
                        v = row.get("success@k", "").strip().lower()
                        if v in ("true", "1"):
                            succ += 1
                            tot += 1
                        elif v in ("false", "0"):
                            tot += 1
                as_map[k] = (succ, tot)
                as_cols.add(f"AS@{k}")

        row = {"Experiment": exp_id}
        row["Loss"] = combine_avg_std(
            metrics.get("Average Best Loss", ""), metrics.get("Std Best Loss", "")
        )
        for avg_key, std_key in TIME_METRICS:
            col = METRIC_KEYS[avg_key]
            row[col] = combine_avg_std(
                metrics.get(avg_key, ""), metrics.get(std_key, "")
            )
        for k, (succ, tot) in as_map.items():
            row[f"AS@{k}"] = f"{succ}/{tot}" if tot else "N/A"
        summary_list.append(row)

    return params_list, summary_list, all_params, as_cols


def sort_and_renumber(params, summaries):
    combined = list(zip(params, summaries))

    def key(item):
        model = item[0].get("model", "").lower()
        grp = 0 if model == "llava" else 1 if model == "gemma" else 2
        num = int(item[0]["Experiment"].replace("exp", ""))
        return grp, num

    combined.sort(key=key)
    for i, (p, s) in enumerate(combined, start=1):
        eid = f"exp{i}"
        p["Experiment"] = s["Experiment"] = eid
    return [p for p, _ in combined], [s for _, s in combined]


def build_latex_table(rows, columns, caption, label):
    lines = [
        r"\begin{table}[ht]",
        r"\centering",
        r"\resizebox{\textwidth}{!}{%",
        "  \\begin{tabular}{" + "l" * len(columns) + "}",
        "  \\hline",
        "  " + " & ".join(latex_escape(c) for c in columns) + r" \\",
        "  \\hline",
    ]
    for r in rows:
        vals = [
            (
                format_numeric(r.get(c, ""), c)
                if caption.startswith("Experiment parameters")
                else latex_escape(r.get(c, ""))
            )
            for c in columns
        ]
        lines.append("  " + " & ".join(vals) + r" \\")
    lines += [
        "  \\hline",
        "  \\end{tabular}",
        "} % end resizebox",
        rf"\caption{{{caption}}}",
        rf"\label{{tab:{label}}}",
        r"\end{table}",
    ]
    return "\n".join(lines)


if __name__ == "__main__":
    params, summaries, all_params, as_columns = collect_rows()
    params, summaries = sort_and_renumber(params, summaries)
    all_params.discard("seed")

    param_cols = ["Experiment"] + sorted(all_params, key=str.lower)
    print("%% ========== TABLE: PARAMETERS ========== %%")
    print(
        build_latex_table(
            params, param_cols, "Experiment parameters.", "experiment_parameters"
        )
    )

    fixed = [
        "Experiment",
        "Loss",
        "Grad (s)",
        "Sampling (s)",
        "PGD (s)",
        "LossTime (s)",
        "Total (s)",
    ]
    as_cols_sorted = sorted(
        as_columns,
        key=lambda x: int(x.split("@")[1]) if x.split("@")[1].isdigit() else x,
    )
    summary_cols = fixed + as_cols_sorted
    print("\n%% ========== TABLE: SUMMARY ========== %%")
    print(
        build_latex_table(
            summaries,
            summary_cols,
            "Summary of experiment results.",
            "experiment_summary",
        )
    )


%% ========== TABLE: PARAMETERS ========== %%
\begin{table}[ht]
\centering
\resizebox{\textwidth}{!}{%
  \begin{tabular}{llllllllllllll}
  \hline
  Experiment & alpha & dynamic\_search & eps & gcg\_attack & joint\_eval & min\_search\_width & model & name & num\_prompts & num\_steps & pgd\_after\_gcg & pgd\_attack & search\_width \\
  \hline
  exp1 & 4/255 & False & 64/255 & False & False & 0 & llava & Llava - PGD Only & 20 & 600 & False & True & 0 \\
  exp2 & 0/255 & False & 0/255 & True & False & 512 & llava & Llava - GCG Only & 20 & 250 & False & False & 512 \\
  exp3 & 4/255 & False & 64/255 & True & False & 0 & llava & Llava - PGD + GCG & 20 & 250 & False & True & 512 \\
  exp4 & 4/255 & False & 64/255 & False & False & 0 & llava & Llava - PGD Only & 10 & 600 & False & True & 0 \\
  exp5 & 0/255 & False & 8/255 & False & False & 0 & llava & Llava - Auto-PGD 2 & 10 & 600 & False & True & 0 \\
  exp6 & 4/255 & False & 64/255 & True & False & 0 & llava & Llava - GCG + PGD & 20 & 250 &

In [2]:
import os
import csv


def latex_escape(s):
    return s.replace("_", r"\_").replace("%", r"\%")


def format_val(v, name=""):
    if v == "True":
        return r"\cmark"
    if v == "False":
        return r"\xmark"
    if name.lower() in {
        "seed",
        "iter",
        "search_width",
        "min_search_width",
        "num_steps",
        "num_prompts",
        "k",
    }:
        try:
            return str(int(float(v)))
        except:
            return latex_escape(v)
    try:
        return f"{float(v):.4f}"
    except:
        return latex_escape(v)


def combo(a, b, d=4):
    try:
        return f"{float(a):.{d}f}±{float(b):.{d}f}"
    except:
        return f"{0:.{d}f}±{0:.{d}f}"


base = "../experiments"
param_rows = []
summary_rows = []
param_keys = set()
as_cols = set()

for idx, fld in enumerate(sorted(os.listdir(base)), start=1):
    path = os.path.join(base, fld)
    if not os.path.isdir(path):
        continue
    exp = f"exp{idx}"
    p = {}
    pf = os.path.join(path, "parameters.csv")
    if os.path.exists(pf):
        with open(pf) as f:
            for r in csv.DictReader(f):
                if r["Parameter"] == "debug_output":
                    continue
                p[r["Parameter"]] = r["Value"]
                param_keys.add(r["Parameter"])
    p["Experiment"] = exp
    param_rows.append(p)

    m = {}
    mf = os.path.join(path, "summary.csv")
    if os.path.exists(mf):
        with open(mf) as f:
            m = {r["Metric"]: r["Value"] for r in csv.DictReader(f)}

    a_map = {}
    er = os.path.join(path, "evaluation")
    if os.path.isdir(er):
        for d in sorted(os.listdir(er)):
            if not d.startswith("evaluation_k"):
                continue
            k = d.split("_k", 1)[1]
            sf = os.path.join(er, d, "summary.csv")
            if not os.path.exists(sf):
                continue
            succ = tot = 0
            with open(sf) as f:
                for r in csv.DictReader(f):
                    v = r.get("success@k", "").strip().lower()
                    if v in {"true", "1"}:
                        succ += 1
                        tot += 1
                    elif v in {"false", "0"}:
                        tot += 1
            a_map[k] = (succ, tot)
            as_cols.add(f"AS@{k}")

    s = {"Experiment": exp}
    s["Loss"] = combo(m.get("Average Best Loss", ""), m.get("Std Best Loss", ""))
    s["Total (s)"] = combo(m.get("Average Total Time", ""), m.get("Std Total Time", ""))
    for k, (su, to) in a_map.items():
        s[f"AS@{k}"] = f"{su}/{to}" if to else "N/A"
    summary_rows.append(s)

param_keys.discard("seed")
combined = list(zip(param_rows, summary_rows))


def sort_key(item):
    m = item[0].get("model", "").lower()
    grp = 0 if m == "llava" else 1 if m == "gemma" else 2
    num = int(item[0]["Experiment"][3:])
    return grp, num


combined.sort(key=sort_key)
for i, (p, s) in enumerate(combined, start=1):
    eid = f"exp{i}"
    p["Experiment"] = s["Experiment"] = eid

param_cols = ["Experiment"] + sorted(param_keys, key=str.lower)
lines = [
    r"\begin{table}[ht]",
    r"\centering",
    r"\resizebox{\textwidth}{!}{%",
    "  \\begin{tabular}{" + "l" * len(param_cols) + "}",
    "  \\hline",
    "  " + " & ".join(latex_escape(c) for c in param_cols) + r" \\",
    "  \\hline",
]
for p, _ in combined:
    vals = [format_val(p.get(c, ""), c) for c in param_cols]
    lines.append("  " + " & ".join(vals) + r" \\")
lines += [
    "  \\hline",
    "  \\end{tabular}",
    "} % end resizebox",
    r"\caption{Experiment parameters.}",
    r"\label{tab:experiment_parameters}",
    r"\end{table}",
]
print("%% ========== TABLE: PARAMETERS ========== %%")
print("\n".join(lines))

fixed = ["Experiment", "Loss", "Total (s)"]
as_sorted = sorted(
    as_cols, key=lambda x: int(x.split("@")[1]) if x.split("@")[1].isdigit() else x
)
final_cols = fixed + as_sorted
hdr = [latex_escape(c) for c in final_cols]
sum_lines = [
    r"\begin{table}[ht]",
    r"\centering",
    r"\resizebox{\textwidth}{!}{%",
    "  \\begin{tabular}{" + "l" * len(hdr) + "}",
    "  \\hline",
    "  " + " & ".join(hdr) + r" \\",
    "  \\hline",
]
for _, s in combined:
    vals = [latex_escape(s.get(c, "")) for c in final_cols]
    sum_lines.append("  " + " & ".join(vals) + r" \\")
sum_lines += [
    "  \\hline",
    "  \\end{tabular}",
    "} % end resizebox",
    r"\caption{Summary of experiment results.}",
    r"\label{tab:experiment_summary}",
    r"\end{table}",
]
print("\n%% ========== TABLE: SUMMARY ========== %%")
print("\n".join(sum_lines))

%% ========== TABLE: PARAMETERS ========== %%
\begin{table}[ht]
\centering
\resizebox{\textwidth}{!}{%
  \begin{tabular}{llllllllllllll}
  \hline
  Experiment & alpha & dynamic\_search & eps & gcg\_attack & joint\_eval & min\_search\_width & model & name & num\_prompts & num\_steps & pgd\_after\_gcg & pgd\_attack & search\_width \\
  \hline
  exp1 & 4/255 & \xmark & 64/255 & \xmark & \xmark & 0 & llava & Llava - PGD Only & 20 & 600 & \xmark & \cmark & 0 \\
  exp2 & 0/255 & \xmark & 0/255 & \cmark & \xmark & 512 & llava & Llava - GCG Only & 20 & 250 & \xmark & \xmark & 512 \\
  exp3 & 4/255 & \xmark & 64/255 & \cmark & \xmark & 0 & llava & Llava - PGD + GCG & 20 & 250 & \xmark & \cmark & 512 \\
  exp4 & 4/255 & \xmark & 64/255 & \xmark & \xmark & 0 & llava & Llava - PGD Only & 10 & 600 & \xmark & \cmark & 0 \\
  exp5 & 0/255 & \xmark & 8/255 & \xmark & \xmark & 0 & llava & Llava - Auto-PGD 2 & 10 & 600 & \xmark & \cmark & 0 \\
  exp6 & 4/255 & \xmark & 64/255 & \cmark & \xmark & 0 & lla